# VGG-16

## Importing Libraries

In [3]:
%matplotlib inline
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

# Device Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Loading the CIFAR-10 Dataset

In [4]:
def load_train_val( data_dir,
                    batch_size,
                    val_batch_size,
                    random_seed,
                    augment,
                    val_size=0.1,
                    shuffle=True):

  normalize = transforms.Normalize(
        mean=[0.4913997551666284, 0.48215855929893703, 0.4465309133731618],
        std=[0.24703225141799082, 0.24348516474564, 0.26158783926049628],
  )

  # Transform
  transform = transforms.Compose([
      transforms.Resize((227,227)),
      transforms.ToTensor(),
      normalize,
  ])

  if augment:
    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])
  else:
    train_transform = transforms.Compose([
        transforms.Resize((227,227)),
        transforms.ToTensor(),
        normalize,
    ])


  # load the dataset
  train_dataset = datasets.CIFAR10(
      root=data_dir, train=True,
      download=True, transform=train_transform,
  )

  val_dataset = datasets.CIFAR10(
      root=data_dir, train=True,
      download=True, transform=transform,
  )

  num_train = len(train_dataset)
  indices = list(range(num_train))
  split = int(np.floor(val_size * num_train))
  if shuffle:
      np.random.seed(random_seed)
      np.random.shuffle(indices)

  train_idx, val_idx = indices[split:], indices[:split]
  train_sampler = SubsetRandomSampler(train_idx)
  val_sampler = SubsetRandomSampler(val_idx)

  train_loader = torch.utils.data.DataLoader(
      train_dataset, batch_size=batch_size, sampler=train_sampler)

  val_loader = torch.utils.data.DataLoader(
      val_dataset, batch_size=val_batch_size, sampler=val_sampler)

  return (train_loader, val_loader)

def load_test(data_dir,
              batch_size,
              shuffle=True):

  normalize = transforms.Normalize(
        mean=[0.4913997551666284, 0.48215855929893703, 0.4465309133731618],
        std=[0.24703225141799082, 0.24348516474564, 0.26158783926049628],
  )

  # Transform
  transform = transforms.Compose([
      transforms.Resize((227,227)),
      transforms.ToTensor(),
      normalize,
  ])

  # load the dataset
  test_dataset = datasets.CIFAR10(
      root=data_dir, train=True,
      download=True, transform=transform,
  )

  data_loader = torch.utils.data.DataLoader(
      test_dataset, batch_size=batch_size, shuffle=shuffle
  )

  return data_loader

In [5]:
# reduced batch size from the original by half cause of memory issues
train_loader, val_loader = load_train_val(data_dir = './data', batch_size = 64, val_batch_size=32, augment=False, random_seed = 1)
test_loader = load_test(data_dir = './data', batch_size = 64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## VGGNET Model

In [6]:
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

## Set HyperParams

In [7]:
num_classes = 10
num_epochs = 25 # 100 took way too long
batch_size = 128
learning_rate = 0.005

# constant for classes
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# Adding weights using normal distribution
def init_weights(model):
  if (isinstance(model, nn.Conv2d) or isinstance(model, nn.Linear)):
    model.weight.data.normal_(0, 0.01)
    model.bias.data.fill_(0.)

model = VGG16(num_classes).to(device)
model.apply(init_weights)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

# Training length
total_step = len(train_loader)

#Tensorboard Setup
tb = SummaryWriter("runs/AlexNet")
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)
tb.add_image("images", grid)

## Training

In [ ]:
total_steps = len(train_loader)

for epoch in tqdm(range(num_epochs)):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #if (epoch+1) % 10 == 0:
    #  print (f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}")\
    
    y_pred = [] # save predction
    y_true = [] # save ground truth
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            y_pred.extend(predicted.data.cpu().numpy())
            y_true.extend(labels.data.cpu().numpy())
            del images, labels, outputs
        #if(epoch+1) % 10 == 0:
        #  print(f"Accuracy of the network on the {5000} validation images: {100*correct/total} %")
    tb.add_scalar("Loss", loss, epoch)
    tb.add_scalar("Correct", correct, epoch)
    tb.add_scalar("Accuracy", correct/total, epoch)
    # Build confusion matrix
    cf_matrix = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index=[i for i in classes],
                         columns=[i for i in classes])
    plt.figure(figsize=(12, 7)) 
    tb.add_figure("Confusion matrix", sn.heatmap(df_cm, annot=True).get_figure(), epoch)
    for name, weight in model.named_parameters():
      tb.add_histogram(name,weight, epoch)
      tb.add_histogram(f'{name}.grad',weight.grad, epoch)

 64%|██████▍   | 16/25 [2:01:02<1:08:07, 454.20s/it]

## Testing

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs
    print(f"Accuracy of the network on the {10000} test images: {100*correct/total} %")

## Tensorboard

In [ ]:
tb.flush()
tb.close()
%load_ext tensorboard
%tensorboard --logdir=runs/AlexNet